In [45]:
import os
import csv
import glob
from tqdm import tqdm
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import numpy as np

In [46]:
csv_root = '/nas/EOS-DB/users/aman/data/RelativePose/frame_data'
pose_gt_dir = '/nas/EOS-DB/users/aman/data/RelativePose/groundtruth_json_poses'
frames_root = '/nas/EOS-DB/users/aman/data/RelativePose/frames'
save_dir = '/nas/EOS-DB/users/aman/data/RelativePose/files'

In [47]:
csv_files = glob.glob(os.path.join(csv_root, '**', '*.csv'), recursive=True)

In [48]:
csv_files

['/nas/EOS/users/aman/data/RelativePose/frame_data/torc/wk1/20210929/20210929.csv',
 '/nas/EOS/users/aman/data/RelativePose/frame_data/torc/wk1/20211001/20211001.csv',
 '/nas/EOS/users/aman/data/RelativePose/frame_data/torc/wk1/20210930/20210930.csv',
 '/nas/EOS/users/aman/data/RelativePose/frame_data/torc/wk2/CAPT-195/CAPT-195.csv',
 '/nas/EOS/users/aman/data/RelativePose/frame_data/torc/wk2/CAPT-205/CAPT-205.csv',
 '/nas/EOS/users/aman/data/RelativePose/frame_data/torc/wk2/CAPT-202/CAPT-202.csv',
 '/nas/EOS/users/aman/data/RelativePose/frame_data/torc/wk2/CAPT-198/CAPT-198.csv',
 '/nas/EOS/users/aman/data/RelativePose/frame_data/torc/wk2/CAPT-192/CAPT-192.csv',
 '/nas/EOS/users/aman/data/RelativePose/frame_data/torc/wk2/CAPT-201/CAPT-201.csv',
 '/nas/EOS/users/aman/data/RelativePose/frame_data/torc/wk2/CAPT-199/CAPT-199.csv',
 '/nas/EOS/users/aman/data/RelativePose/frame_data/torc/wk2/CAPT-194/CAPT-194.csv',
 '/nas/EOS/users/aman/data/RelativePose/frame_data/torc/wk3/CAPT-357/CAPT-35

In [19]:
len(csv_files)

45

Number of sequences = 45

In [20]:
data = [('continental',lambda x : True),
        ('torc',lambda x : x =='wk1'),
        ('torc',lambda x : x =='wk2'),
        ('torc',lambda x : x =='wk3'),
        ('torc',lambda x : True),
        ('all',lambda x : True)]

In [6]:
train_val_data = []

for csv_file in tqdm(csv_files):
    df = pd.read_csv(csv_file, header=None)
    for indx, row in df.iterrows():
        root, capture, sequence, cam_type, left_id, right_id = row
        dataset = 'torc' if 'torc' in root else 'continental'

        if dataset == 'torc':
            week = root.split('/')[-2]
        else:
            week = ''

        left_img_subpath = left_id + '.png'
        right_img_subpath = right_id + '.png'

        left_cam_num = left_id.split('/')[0][-1]
        right_cam_num = right_id.split('/')[0][-1]

        left_img_path = os.path.join(frames_root,dataset,str(capture),sequence,cam_type,left_img_subpath)
        right_img_path = os.path.join(frames_root,dataset,str(capture),sequence,cam_type,right_img_subpath)

        if os.path.isfile(left_img_path) and os.path.isfile(right_img_path) and os.path.getsize(left_img_path) != 0 and os.path.getsize(right_img_path) != 0:
            train_val_data.append([dataset,week,capture,cam_type,left_cam_num,right_cam_num,os.path.join(dataset,str(capture),sequence,cam_type,left_img_subpath),
                                os.path.join(dataset,str(capture),sequence,cam_type,right_img_subpath)])


100%|██████████| 45/45 [14:18<00:00, 19.08s/it]


In [21]:
len(train_val_data)

67503

In [22]:
df = pd.DataFrame(train_val_data,
 columns =['dataset', 'week', 'capture', 'cam_type', 'left_cam_num', 'right_cam_num', 'left_img_path', 'right_img_path'])

---

In [38]:
def filter_dataframe(dataframe, dataset = None,week = None,save_dir=save_dir, test_size = None):
    if dataset is not None and week is not None:
        filt_df = dataframe[(df['dataset'] == dataset) & (df['week'] == week)]
        fname = dataset + '_' + week + '.csv'
        subdir = dataset
    elif week is None and dataset is not None:
        filt_df = dataframe[df['dataset'] == dataset]
        fname = dataset + '.csv'
        subdir = dataset
    else:
        filt_df = dataframe
        fname = 'dataset.csv'
        subdir = ''
    os.makedirs(os.path.join(save_dir,subdir),exist_ok=True)
    filt_df.to_csv(os.path.join(save_dir,subdir,fname), header=False, index=False)

    if test_size is not None:
        train, val = train_test_split(filt_df, test_size=test_size)
        os.makedirs(os.path.join(save_dir,"splitfiles",subdir),exist_ok=True)
        train.to_csv(os.path.join(save_dir,"splitfiles",subdir,"train_"+fname),
                     header=False,
                     index=False)
        val.to_csv(os.path.join(save_dir,"splitfiles",subdir,"val_"+fname),
                   header=False,
                   index=False)

In [44]:
filter_dataframe(df,dataset = 'torc', week='wk3', test_size=0.2)

In [34]:
right_img_path

'/nas/EOS/users/aman/data/RelativePose/frames/continental/CAR-299/highway_03-03-2021_07h00m03/fsc231/image1/frame_42378_1614798747666642_2832405229120.png'

In [35]:
right_id.split('/')[0][-1]

'1'

In [36]:
len(train_val_data)

10946

In [37]:
train_val_data

[['continental',
  '',
  'CAR-359',
  'fsc231',
  '0',
  '1',
  'continental/CAR-359/rotative1_25-03-2021_07h36m33/fsc231/image0/frame_35854_1616701024295364_3592389718760.png',
  'continental/CAR-359/rotative1_25-03-2021_07h36m33/fsc231/image1/frame_35854_1616701024287291_3592389831490.png'],
 ['continental',
  '',
  'CAR-359',
  'fsc231',
  '0',
  '1',
  'continental/CAR-359/rotative1_25-03-2021_07h36m33/fsc231/image0/frame_35686_1616701007484801_3575579954670.png',
  'continental/CAR-359/rotative1_25-03-2021_07h36m33/fsc231/image1/frame_35686_1616701007488839_3575580015870.png'],
 ['continental',
  '',
  'CAR-359',
  'fsc231',
  '0',
  '1',
  'continental/CAR-359/rotative1_25-03-2021_07h36m33/fsc231/image0/frame_35697_1616701008591958_3576689637805.png',
  'continental/CAR-359/rotative1_25-03-2021_07h36m33/fsc231/image1/frame_35697_1616701008587039_3576689560105.png'],
 ['continental',
  '',
  'CAR-359',
  'fsc231',
  '0',
  '1',
  'continental/CAR-359/rotative1_25-03-2021_07h36m33/

In [38]:
with open('/home/amanpreet.walia/workspace/code/RelativePoseEstimation/src/data/split_files/continental/dataset.csv','w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(train_val_data)

In [39]:
frames_root,dataset,capture,sequence,cam_type,left_img_subpath

('/nas/EOS/users/aman/data/RelativePose/frames',
 'continental',
 'CAR-299',
 'highway_03-03-2021_07h00m03',
 'fsc231',
 'image0/frame_42378_1614798747661022_2832405166925.png')

In [40]:
os.path.join('frames','','dir')

'frames/dir'

In [41]:
data

NameError: name 'data' is not defined

In [42]:
train_data, val_data = train_test_split(train_val_data, test_size=0.2, random_state=42)

In [43]:
with open('/home/amanpreet.walia/workspace/code/RelativePoseEstimation/src/data/split_files/continental/train_dataset.csv','w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(train_data)

In [44]:
with open('/home/amanpreet.walia/workspace/code/RelativePoseEstimation/src/data/split_files/continental/val_dataset.csv','w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(val_data)

In [17]:
pose_gt = {}
        
# scan for all the json files in the pose groundtruth dir
json_fpaths = sorted(glob.glob(os.path.join(pose_gt_dir,'**','*.json'), recursive=True))

In [18]:
for json_fpath in json_fpaths:
    dataset = 'torc' if 'torc' in json_fpath else 'continental'
    week = json_fpath.split('/')[-2] if dataset == 'torc' else ''
    dataset_key = f'{dataset}_{week}' if len(week) > 0 else dataset 
    if dataset_key not in pose_gt.keys():
        pose_gt[dataset_key] = {}
    with open(json_fpath,'r') as fh:
        pose_gt[dataset_key].update(json.load(fh))


In [78]:
pose_gt

{'continental': {'CAR-224': {'fsc231': {'extrinsics': {'0_1': {'rmat': [0.9998352261878228,
       -0.002475334693767253,
       -0.017983136325432674,
       0.002317042594423979,
       0.999958438333642,
       -0.008817761561738292,
       0.01800421582743578,
       0.008774640932703543,
       0.9997994068256598],
      'tvec': [-0.8634172398868871,
       0.006463608596794268,
       -0.002461995576343028]}},
    'intrinsics': {'0': {'K': [7656.892578125,
       0.0,
       1723.0555419921875,
       0.0,
       7686.619140625,
       1000.4788818359375,
       0.0,
       0.0,
       1.0],
      'dist': [0.0,
       0.0,
       0.0,
       0.0,
       0.0,
       0.0,
       0.0,
       0.0,
       0.0,
       0.0,
       0.0,
       0.0,
       0.0,
       0.0]},
     '1': {'K': [7656.892578125,
       0.0,
       1723.0555419921875,
       0.0,
       7686.619140625,
       1000.4788818359375,
       0.0,
       0.0,
       1.0],
      'dist': [0.0,
       0.0,
       0.0,
  

In [88]:
pose = pose_gt['torc_wk1']['20210929']['ar0820']['extrinsics']['0_1']
rmat, tvec = np.array(pose['rmat']).reshape(3,3),np.array(pose['tvec']).reshape(3,1)

In [89]:
rmat

array([[ 0.99989614,  0.00190678, -0.01428535],
       [-0.0018166 ,  0.99997836,  0.00632349],
       [ 0.0142971 , -0.00629688,  0.99987796]])

In [90]:
import random

In [93]:
random.random()

0.017766931954109166

In [94]:
a = {'1':'apple','2':'boy'}

In [95]:
def m(dict_data):
    for k in list(dict_data):
        dict_data[k+'0'] = None

In [96]:
m(a)

In [97]:
a

{'1': 'apple', '2': 'boy', '10': None, '20': None}

In [98]:
a

{'1': 'apple', '2': 'boy', '10': None, '20': None}

In [8]:
import cv2

In [21]:
os.path.getsize('/nas/EOS-DB/users/aman/data/RelativePose/frames/torc/CAPT-411/CAPT-411_dusk_20211215175934/ar0820/image0/frame_image0_00048444.png')

0

In [11]:
x == None

True

In [6]:
a = "True"

In [7]:
bool(a)

True

In [8]:
a == "True"

True